In [322]:
import math 
import random

In [336]:
class Value:
    def __init__(self, data, _children=()):
        self.data=data
        self.grad=0.
        self._backward = lambda: None
        self._prev = _children

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other))

        def _backward():
            self.grad += 1. * out.grad
            other.grad += 1. * out.grad
        out._backward = _backward
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other))
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int and float power for now"
        out = Value(self.data**other, (self,))
        def _backward():
            self.grad+=other * self.data**(other-1) * out.grad
        out._backward = _backward
        return out

    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x)+1)
        out = Value(t, (self, ))
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out
    
    def exp(self):
        x = self.data
        t = math.exp(x)
        out = Value(t, (self, ))
        def _backward():
            self.grad+=t * out.grad
        out._backward = _backward
    
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        self.grad=1.
        build_topo(self)
        for node in reversed(topo):
            node._backward()

    def __rmul__(self, other):
        return self*other
    
    def __radd__(self, other):
        return self + other
    
    def __truediv__(self, other):
        return self * other**-1
    
    def __neg__(self):
        return self * -1
    
    def __sub__(self, other):
        return self + (-other)
    
    def __rsub__(self, other):
        return other + (-self)
    
    def __rmul__(self, other):
        return self*other
    
    def __rtruediv__(self, other):
        return  other * self**-1

    def __repr__(self):
        return f"Value(data={self.data} ; grad={self.grad})"

In [337]:
class Neuron:
    def __init__(self, nin, nonlin=True):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(0)
        self.nonlin=nonlin
    
    def __call__(self, x):
        act = sum((wi*xi for wi, xi in zip(self.w, x)), start=self.b)
        act = act.tanh() if self.nonlin else act
        return act
    
    def parameters(self):
        return self.w + [self.b]
    
    def __repr__(self):
        return f"{"Tanh" if self.nonlin else "Linear"}Neuron({len(self.w)})"
    

class Layer:
    def __init__(self, nin, nout, nonlin=True):
        self.neurons = [Neuron(nin, nonlin=nonlin) for _ in range(nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs)==1 else outs
    
    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]
    
    def __repr__(self):
        return f"Layer of [{', '.join(str(n) for n in self.neurons)}]"
    
class MLP:
    def __init__(self, nin, nouts):
        sz = [nin]+nouts
        self.layers = [Layer(sz[i], sz[i+1], nonlin=i!=(len(nouts)-1)) for i in range(len(nouts))]
        
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for l in self.layers for p in l.parameters()]
    
    def __repr__(self):
        return f"MLP of {', '.join(str(layer) for layer in self.layers)}"

In [341]:
mlp = MLP(3, [4, 4, 1])

xs = [
    [2., 3., -1.],
    [3., -1., 0.5],
    [0.5, 1., 1.],
    [1., 1., -1.]
]
ys = [1., -1., -1., 1.]

for _ in range(20):
    ypred = [mlp(x) for x in xs]

    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))

    for p in mlp.parameters():
        p.grad=0
    loss.backward()
    for p in mlp.parameters():
        p.data += -0.05*p.grad
    print(_, loss.data)

0 4.813873373094015
1 2.4508973381134607
2 1.2466908815892879
3 0.5610056163032613
4 0.23468326425418706
5 0.11296872247594329
6 0.06749090290558696
7 0.04612871878562403
8 0.03389026119404875
9 0.026144399934876106
10 0.02091954352020147
11 0.017186027424041333
12 0.014370124366006515
13 0.012147669515345887
14 0.0103328928781748
15 0.008816388037595515
16 0.007530848348059912
17 0.006432249358137108
18 0.005489687032779701
19 0.004679891019390758
